In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException
import time
import re
total_review= []
visited_urls = set()
url_pattern = r"https?:\/\/.*?\.html|\/\/.*?\.html"
options = Options()
options.add_argument("--no-sandbox") 
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

url = "https://www.daraz.com.np/products/apple-iphone-16-pro-max-69-inch-super-retina-xdr-oled-display-a18-chip-pro-camera-system-i164897694-s1181305969.html"
visited_urls.add(url)
# Load page twice (key part that works)
driver.get(url)
wait = WebDriverWait(driver, 3)
wait.until(lambda d: d.execute_script("return document.readyState") == "complete")

driver.get(url)  # Second load triggers content
wait.until(lambda d: d.execute_script("return document.readyState") == "complete")
driver.maximize_window()
# User interaction
actions = ActionChains(driver)
actions.move_by_offset(100, 100).perform()
driver.execute_script("""window.scrollTo({top: 1300,behavior: 'smooth'});""")

time.sleep(1)
total_review.append(get_review(driver))
driver.execute_script("window.scrollTo({top: document.body.scrollHeight - 300, behavior: 'smooth'});")
for i in range(0):
    try:
        for i in range(3):
            driver.execute_script(f"window.scrollTo({{top: {1000 + i*800}, behavior: 'smooth'}});")
            time.sleep(2)  # Wait for lazy loading
        next_item_mark =  driver.find_element(By.CSS_SELECTOR, 'a[data-spm-anchor-id="a2a0e.pdp_revamp.recommendation_2.1"]')
        driver.execute_script("""arguments[0].scrollIntoView({ behavior: 'smooth', block: 'center' });""", next_item_mark)
        driver.execute_script(""" window.scrollBy({top: 800,behavior: 'smooth'});""")
        for j in range(4):
            next_item= driver.find_element(By.CSS_SELECTOR, f'a[data-spm-anchor-id="a2a0e.pdp_revamp.recommendation_2.{j+1}"]')
            match = re.search(url_pattern, next_item.get_attribute("href"))
            if match:
                clean_href = "https:" + match.group() if match.group().startswith("//") else match.group()
                if clean_href not in visited_urls:
                    visited_urls.add(clean_href)
                    next_item.click()
                    wait.until(lambda d: d.execute_script("return document.readyState") == "complete")
                    actions.move_by_offset(100, 100).perform()
                    driver.execute_script("""window.scrollTo({top: 3300,behavior: 'smooth'});""")
                    total_review.append(get_review(driver))
                    break
            else:
                print("No match found.")
    except Exception as e:
        print(e)
driver.quit()

[<selenium.webdriver.remote.webelement.WebElement (session="f448e1f568a333450e876610ef2cf368", element="f.7A8AA18A93FF40DCCE2F3C4DB304C99A.d.858B7AC94B04A2DBDCE9FD7EE144F578.e.136")>, <selenium.webdriver.remote.webelement.WebElement (session="f448e1f568a333450e876610ef2cf368", element="f.7A8AA18A93FF40DCCE2F3C4DB304C99A.d.858B7AC94B04A2DBDCE9FD7EE144F578.e.137")>, <selenium.webdriver.remote.webelement.WebElement (session="f448e1f568a333450e876610ef2cf368", element="f.7A8AA18A93FF40DCCE2F3C4DB304C99A.d.858B7AC94B04A2DBDCE9FD7EE144F578.e.138")>, <selenium.webdriver.remote.webelement.WebElement (session="f448e1f568a333450e876610ef2cf368", element="f.7A8AA18A93FF40DCCE2F3C4DB304C99A.d.858B7AC94B04A2DBDCE9FD7EE144F578.e.139")>, <selenium.webdriver.remote.webelement.WebElement (session="f448e1f568a333450e876610ef2cf368", element="f.7A8AA18A93FF40DCCE2F3C4DB304C99A.d.858B7AC94B04A2DBDCE9FD7EE144F578.e.140")>, <selenium.webdriver.remote.webelement.WebElement (session="f448e1f568a333450e876610e

In [22]:
def get_review(driver):
    review_list=[]
    try:
        review = driver.find_elements(By.CSS_SELECTOR, "div.content")
        print(review)
        for index, item in enumerate(review):
            if index % 2 == 1:
                review_list.append(item.text)
            
    except Exception as e:
        print(f"Error finding value{e}")
    #finding items review    
    while True:
        try:
            time.sleep(4)
            WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.content")))
            review_section = driver.find_element(By.CSS_SELECTOR,"div.pdp-mod-review[data-spm='ratings_reviews']")
            next_button = driver.find_element(By.CSS_SELECTOR, ".next-btn.next-btn-normal.next-btn-medium.next-pagination-item.next")
            is_inside = driver.execute_script("""return arguments[1].contains(arguments[0]);""", next_button, review_section)
            #Checking if the next button is either disabled or not of the review section
            if next_button.get_attribute("disabled") or not is_inside:
                raise NoSuchElementException("Next button is disabled.")
            
            
                    
            driver.execute_script("""arguments[0].scrollIntoView({ behavior: 'smooth', block: 'center' });""", next_button)
            driver.execute_script(""" window.scrollBy({top: +800,behavior: 'smooth'});""")
            
            #clicks if safe 
            if not safe_click(driver, next_button):
                print("Failed to click after all attempts")
                continue
            time.sleep(2)

            review = driver.find_elements(By.CSS_SELECTOR, "div.content")
            for index, item in enumerate(review):
                if index % 2 == 1:
                    review_list.append(item.text)
                
        except NoSuchElementException:
            print("No next button found. Stopping.")
            break
    return review_list

In [23]:
def safe_click(driver, element, max_attempts=3):
    """Safely click an element with multiple fallback strategies"""
    for attempt in range(max_attempts):
        try:
            element.click()
            return True
        except ElementClickInterceptedException:
            print(f"Click intercepted, attempt {attempt + 1}")
            
            if attempt == 0:
                # First attempt: scroll element into better view
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", element)
                time.sleep(1)
            elif attempt == 1:
                # Second attempt: scroll a bit more
                driver.execute_script("window.scrollBy({top: -200, behavior: 'smooth'});")
                time.sleep(1)
            else:
                # Final attempt: JavaScript click as last resort
                driver.execute_script("arguments[0].click();", element)
                return True
                
    return False

In [7]:
visited_urls

{'https://www.daraz.com.np/products/asus-zenbook-14-oled-intel-evo-ultra-5-125h-built-in-ai-8gb-ddr5-ram-512gb-nvme-gen-4-ssd-intel-arc-graphics-14-wuxga-oled-touch-display-backlit-keyboard-i146148668-s1095972230.html',
 'https://www.daraz.com.np/products/lenovo-ideapad-slim-3-intel-core-i5-13th-gen-13420h-16gb-ddr5-ram-512gb-nvme-ssd-intel-integrated-graphics-156-fhd-display-arctic-grey-i128471375-s1035960584.html',
 'https://www.daraz.com.np/products/lenovo-ideapad-slim-3-intel-i7-13th-gen-13620h-16gb-ddr5-ram-512gb-nvme-ssd-intel-uhd-integrated-graphics-156-fhd-display-arctic-grey-i164153640-s1174437054.html'}

In [28]:
len(total_review[2])

5

In [27]:
total_review

[['Got my phone same day of the order(ordered midnight) genuine phone, genuine seller 👍 definitely go for it!!!',
  'Very best product at best price',
  'dami xa']]

In [17]:
len(total_review[1])

5

In [ ]:
<div class="content" data-spm-anchor-id="a2a0e.pdp_revamp.ratings_reviews.i0.276a20f0jHf5El">
<div class="content" data-spm-anchor-id="a2a0e.pdp_revamp.ratings_reviews.i1.276a20f0jHf5El">
<div class="content" data-spm-anchor-id="a2a0e.pdp_revamp.ratings_reviews.i4.276a20f0jHf5El">
<div class="content" data-spm-anchor-id="a2a0e.pdp_revamp.ratings_reviews.i6.276a20f0jHf5El">I love the products quality is so good</div>
<div class="content" data-spm-anchor-id="a2a0e.pdp_revamp.ratings_reviews.i2.4f864ed9H59zMz">suruko 1 months jati ramrai chalyo tespaxi ghari right side matra sound aaune ghari left side matra sound aaune hunxa. Durable chai na raixa. </div>
a2a0e.pdp_revamp.ratings_reviews.i0.4f864ed9H59zMz

In [36]:
import re
href="//www.daraz.com.np/products/fantech-hq53-lightweight-gaming-headset-lightweight-design-with-red-accent-lighting-i118894138-s1032560386.html?&amp;search=pdp_v2v?mp=1&amp;scm=1007.38553.419544.0&amp;clickTrackInfo=8b44c717-2cc2-499e-abb9-02d8efc46d58__118894138__10002907__hero_item_i2i__419526__0.0__0.04698380082845688__0.0__0.0__0.0__0.0__0____128117636__128117636__0.0________2799.0__0.28581636298678104__4.265129682997118__347__1999.0__1969400,1967665,1970357,1244135,1985972,1987155,1981175,1960436,1196930,1960437,1683891,338926,338928,1978238,1972330,534814,808110,1797945,1797946,153780,1243932,1213024,1975634,294830,390289,1955329,332334,759053,680013,1952971,12005,1957386,806100,1176891,1364821,176376,323495,1963322,1379425,1965068,1960492,1467971,339834,1960457,1955255,210928,1988626,1684884,1790813,1199863,1957393,1970373,1789909,1967672,295703,350723,1835137,1835138,1979419,537206,1967711,1971578,1960483,1965138,1988555,1519067,1838122,1242700,1985569,1957368,1960569,1149832,470812,1955348,1960522,1682899,1977794,515824____________32104____0.0__0.0__0.0__0.0____0.0____0.0__0.0__0.0____________0.0" 
pattern = r"https?:\/\/.*?\.html|\/\/.*?\.html"
a = re.match(pattern,href)
print(a.group())

//www.daraz.com.np/products/fantech-hq53-lightweight-gaming-headset-lightweight-design-with-red-accent-lighting-i118894138-s1032560386.html


In [39]:
a

[['a']]

In [ ]:
a2a0e.pdp_revamp.recommendation_2.1